In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torch.utils.data as utils_Data

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
# Load MNIST data
from torchvision import datasets, transforms 

transforms = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])

trainset = datasets.MNIST('./data', train=True, download=True,
                          transform=transforms)

train_loader = utils_Data.DataLoader(trainset, batch_size=100, shuffle=True)

testset = datasets.MNIST('./data', train=False, download=True,
                         transform=transforms)

test_loader = utils_Data.DataLoader(testset, batch_size=100, shuffle=True)

In [7]:
model = Net()

use_cuda = False
if(use_cuda):
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx,(data, label) in enumerate(train_loader):
        if (use_cuda):
            data, lable = data.cuda(), label.cuda()
        data, label = Variable(data), Variable(label)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, label in test_loader:
        if (use_cuda):
            data, lable = data.cuda(), label.cuda()
        data, label = Variable(data, volatile=True), Variable(label)
        output = model(data)
        test_loss += F.nll_loss(output, label).data[0]
        pred = output.data.max(1)[1]
        correct += pred.eq(label.data).cpu().sum()
        
    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
#for epoch in range(1, 10):
train(1)
test(1)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.282120


Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.076413


Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.353708


Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.456998


Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.465649


...

Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.186215


Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.148934


Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.241503


Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.119649



Test set: Average loss: 0.1195, Accuracy: 9634/10000 (96%)



In [9]:
import numpy as np
params = list(model.parameters())
len(params)
conv1 = params[0]
type(conv1)
np_conv1 = conv1.data.numpy()
print(conv1[0])
print(np_conv1[0])
#np_weight = params[0][0].numpy()
#print (np_weight)

Variable containing:
(0 ,.,.) = 
 -0.1470  0.1724  0.2129 -0.0837  0.1885
 -0.0638 -0.0006  0.1741  0.2042  0.0536
 -0.1910  0.0705  0.0155 -0.1089  0.0683
  0.1090 -0.1169 -0.1627 -0.1711  0.1138
 -0.2234  0.0195 -0.0597 -0.0761 -0.1781
[torch.FloatTensor of size 1x5x5]

[[[-0.14699253  0.1723911   0.21292394 -0.08366029  0.18851154]
  [-0.06378265 -0.00061548  0.17408545  0.20421942  0.05361126]
  [-0.19102345  0.07054869  0.01552966 -0.10892844  0.06830549]
  [ 0.10896599 -0.11694469 -0.1626578  -0.17106026  0.11381786]
  [-0.22340022  0.01953809 -0.05969426 -0.07605883 -0.17809737]]]


In [20]:
np.save('a', np_conv1)

In [16]:
file = open('weight', 'w')
print np_conv1[0][0][0]
file.write(np_conv1[0][0][0])
file.close()

[-0.14699253  0.1723911   0.21292394 -0.08366029  0.18851154]
